### Example 1: 2 node system

#### Importing packages

In [567]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import cartopy as ccrs

In [568]:
n = pypsa.examples.scigrid_de(from_master=True)

INFO:pypsa.io:Imported network scigrid-de.nc has buses, generators, lines, loads, storage_units, transformers


#### Step 1:Fuel data, Powerplant efficiencies & Specific emissions

In [569]:
#A) fuel costs in € / MWh$_{th}$ 
fuel_cost = {"coal": 30, "oil": 50, "gas": 55}

In [570]:
# B) efficiencies of thermal power plants in MWh$_{el}$ / MWh$_{th}$
efficiency = {"coal": 0.33, "oil": 0.35, "gas": 0.58}

In [571]:
# C) Specific emissions are in t/MWh thermal
emissions = dict(
    coal=0.34, 
    gas=0.2, 
    oil=0.26,
)

In [572]:
# D) Power plant capacities in MWp
power_plants = {"coal": 35000, "oil": 2000, "gas": 8000}

In [573]:
# E) Add load in MW
loads=40000

#### Step 2: Building Basic network

In [574]:
# A) Create a network object
n=pypsa.Network()

In [575]:
# B) Add buses
n.add("Bus", "Bus_1", y=128.19736905, x= -3.67176883, v_nom=400, carrier='AC')#load

n.add("Bus", "Bus_2", y=128.30808593, x= -3.60694618, v_nom=400, carrier='AC') #powerplant

In [576]:
# C) Add carriers with emissions associated with it
n.madd(
    "Carrier",
    ["coal", "gas", "oil"], 
    co2_emissions=emissions,
    nice_name=["Coal", "Gas", "Oil"],
    color=["black", "indianred", "grey"],
) 

Index(['coal', 'gas', 'oil'], dtype='object')

In [577]:
# D) Add generators in loop for type of powerplants mentioned already 
for tech, capacity in power_plants.items(): #Using .items() to access key-value pairs
    n.add("Generator",
        f"PP{tech}",
        bus="Bus_2",
        carrier = tech,
        efficiency = efficiency.get(tech, 1),
        p_nom=capacity,
        marginal_cost=fuel_cost.get(tech, 0) / efficiency.get(tech, 1),
)

In [578]:
n.generators

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
PPcoal,Bus_2,PQ,,35000.0,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
PPoil,Bus_2,PQ,,2000.0,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
PPgas,Bus_2,PQ,,8000.0,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [579]:
# E) Add electricity demand
n.add("Load",
    "Ambon_elec_demand",
    bus="Bus_2",
    p_set=loads,
    carrier='electricity',
)

In [580]:
# F) Add line connection between the buses
n.add("Line",
    "Bus_1-Bus_2",
    bus0="Bus_1",
    bus1="Bus_2",
    s_nom=500,
    x=1,
)

In [581]:
n.lopf(solver_name='glpk')

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.07s
INFO:pypsa.linopf:Solve linear problem using Glpk solver
INFO:pypsa.linopf:Optimization successful. Objective value: 3.66e+06


('ok', 'optimal')

In [582]:
n.generators_t.p

Generator,PPcoal,PPoil,PPgas
snapshot,,,
now,35000.0,0.0,5000.0


In [583]:
n.buses_t.marginal_price

Bus,Bus_1,Bus_2
snapshot,,
now,94.8276,94.8276


In [586]:
s = n.loads.groupby('bus').p_set.sum() / 1e4

In [590]:
n.generators_t.p.loc["now"]

Generator
PPcoal    35000.0
PPoil         0.0
PPgas      5000.0
Name: now, dtype: float64

In [591]:
s = n.generators_t.p.loc["now"].groupby(
    [n.generators.bus, n.generators.carrier]
).sum()

In [592]:
s

bus    carrier
Bus_2  coal       35000.0
       gas         5000.0
       oil            0.0
Name: now, dtype: float64